# Lab | Customer Analysis Round 7

In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pandas.plotting import scatter_matrix
import seaborn as sns
from IPython.display import set_matplotlib_formats, HTML
from matplotlib.dates import DateFormatter
import matplotlib_inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import colors as mcolors
from pandas.plotting import register_matplotlib_converters
import plotly.express as px
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore') 

In [10]:

from sklearn.metrics import r2_score

# a lot of different scaler to try
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer, PolynomialFeatures

# a lot of different regression models to try
from sklearn.linear_model import LinearRegression, HuberRegressor, RANSACRegressor
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge

# more regression models
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor 

In [8]:
# Formating Plots
# default styles
def set_sns_format(width=14, height=8):
    sns.set_theme(palette='pastel', context='notebook',rc={'savefig.dpi':300})
    matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
    matplotlib.rcParams['figure.figsize'] = (width, height)
    return None
set_sns_format(width=14, height=8)

In [ ]:
def add_value_labels(ax, typ, spacing=5):
    #This function add the labels in the bar and line plots
    #input the ax to add the labels, the type of plot
    
    space = spacing
    va = 'bottom'
    

    if typ == 'bar':
        for i in ax.patches:
            y_value = i.get_height()
            x_value = i.get_x() + i.get_width() / 2

            label = "{:.2f}".format(y_value)
            ax.annotate(label,(x_value, y_value), xytext=(0, space), 
                    textcoords="offset points", ha='center', va=va, fontsize=10)     

    if typ == 'line':
        for line in ax.lines:
            for x_value, y_value in zip(line.get_xdata(), line.get_ydata()):
                label = "{:.2f}".format(y_value)
                ax.annotate(label,(x_value, y_value), xytext=(0, space), 
                    textcoords="offset points", ha='center', va=va, fontsize=10)

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [11]:
data = pd.read_csv(r'C:\Users\ssai\OneDrive\Data_26-07\labs\Lab-Customer-Analysis-round-7\lab-customer-analysis-round-7\files_for_lab\csv_files/marketing_customer_analysis.csv')

In [12]:
poly = PolynomialFeatures()
use_poly = False
reg =  LinearRegression()                #initialization of the method
numerical_transformer = StandardScaler() #initialization of the method
split_feature = 'total_claim_amount'     # choose feature to split
train_test_split_ratio = 0.2             # ratio for train and test split
outlier = ['number_of_policies']         # columns to be cleaned of outliers

# list of features/columns that have low coefficient values for the Standard model and will be dropped
drop_columns = ['customer', 'effective_to_date', 'sales_channel', 'state', 'education', 'number_of_open_complaints'
               , 'months_since_policy_inception', 'vehicle_size', 'policy',
                'months_since_last_claim']

In [13]:
data.columns = data.columns.str.lower()             # transform all column names to lower case
data.columns = data.columns.str.replace(' ','_')    # repalce spaces inbetween with '_'

Q1 = data.quantile(0.25) # first quantile
Q3 = data.quantile(0.75) # third quantile
IQR = Q3 - Q1            # inter quantile range

data = data[~(                     # negation so we get the datapoints within the whiskers
(data[outlier] < (Q1 - 1.5 * IQR)) # datapoints left of the "left whisker"
|(data[outlier] > (Q3 + 1.5 * IQR) # datapoints right of the 'right whisker'
)).any(axis=1)]

data = data.drop(columns=drop_columns) # dropping low impact columns

In [14]:
x = data[data.columns.drop(split_feature)] # features
y = data[split_feature]                    # target feature
x = pd.get_dummies(x, drop_first = True)   # dummification

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = train_test_split_ratio, random_state = 42) # train test split

if use_poly:
    X_train = poly.fit_transform(X_train) # fit and transform X_train with PolynomialFeatures
    X_test = poly.transform(X_test)       # transformX_test with PolynomialFeatures
else:
    X_train = numerical_transformer.fit_transform(X_train) # fit and transform X_train
    X_test = numerical_transformer.transform(X_test)       # transformX_test

In [16]:
reg.fit(X_train, y_train) # train on the train data

LinearRegression()

In [18]:
predictions_train = reg.predict(X_train) # create predictions for our train data
predictions_test = reg.predict(X_test)   # create predictions for our test data

In [19]:
r2_train = r2_score(y_train, predictions_train) # calculate r2 score for train data
r2_test = r2_score(y_test, predictions_test)    # calculate r2 score for test data

print('R2 value for train: {}'.format(r2_train))
print('R2 value for test: {}'.format(r2_test))  

R2 value for train: 0.7696091117258597
R2 value for test: 0.7716572809167026


In [20]:
# safe scores from previous attempts in a dictionary
rms = {'Model used' : ['LinearRegression', 'HuberRegressor', 'RANSACRegressor', 'ElasticNet', 
                       'SGDRegressor', 'BayesianRidge', 'SVR', 'KernelRidge', 'GradientBoostingRegressor'],
    'Train score' : [0.77, 0.75, 0.66, 0.66, 0.76, 0.76, 0.31, -1.4, 0.86],
    'Test score' : [0.77, 0.73, 0.67, 0.68, 0.74, 0.74, 0.32, -1.5, 0.82],
    'Train score using PolynomialFeatures' : [0.86, -0.25, 0.42, 0.84, -3, 0.83, 0.08, 0.59, 0.88],
    'Test scote using PolynomialFeatures' : [0.80, -0.25, 0.42, 0.82, -3, 0.81, 0.09, 0.46, 0.82],
}

In [21]:
regression_model_scores = pd.DataFrame(data=rms)
regression_model_scores

,Model used,Train score,Test score,Train score using PolynomialFeatures,Test scote using PolynomialFeatures
0,LinearRegression,0.77,0.77,0.86,0.80
1,HuberRegressor,0.75,0.73,-0.25,-0.25
2,RANSACRegressor,0.66,0.67,0.42,0.42
3,ElasticNet,0.66,0.68,0.84,0.82
4,SGDRegressor,0.76,0.74,-3.00,-3.00
5,BayesianRidge,0.76,0.74,0.83,0.81
6,SVR,0.31,0.32,0.08,0.09
7,KernelRidge,-1.40,-1.50,0.59,0.46
8,GradientBoostingRegressor,0.86,0.82,0.88,0.82


### Conclussion

The best scores are obtained with GradienBoostingREgressor with PolynomialFeatures 